This notebook contains an example for teaching.

# The Effect of Gun Ownership on Gun-Homicide Rates - proceeding

In this lab, we estimate the effect of gun ownership on the homicide rate by a neural network.

In [55]:
using Pkg
Pkg.add("CSV"), using CSV
Pkg.add("DataFrames"), using DataFrames
Pkg.add("StatsModels"), using StatsModels
Pkg.add("GLM"), using GLM
Pkg.add("Random"), using Random
using Statistics, Plots, FixedEffectModels, MLDataUtils, MLBase

   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


First, we need to load and preprocess the data.

In [77]:
data = CSV.File("../data/gun_clean.csv") |> DataFrame;
println("Number of rows: ",size(data,1))
println("Number of columns: ",size(data,2))

Number of rows: 3900
Number of columns: 415


In [78]:
################################# Create Variables ###############################

# Dummy Variables for Year and County Fixed Effects
fixed = filter(x->contains(x, "X_Jfips"), names(data));
year = filter(x->contains(x, "X_Tyear"), names(data));

In [79]:
census = []
census_var = ["AGE", "BN", "BP", "BZ", "ED", "EL", "HI", "HS", "INC", "LF", "LN", "PI", "PO", "PP", "PV", "SPR", "VS"]

for i in 1:size(census_var,1) 
    append!(census, filter(x->contains(x, census_var[i]), names(data)))
end

In [80]:
################################ Variables ##################################

# Treatment Variable
d = ["logfssl"];

# Outcome Variable
y = ["logghomr"];

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"];
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"];

In [81]:
#################################  Partial out Fixed Effects ########################

# Variables to be Partialled-out
variable = [y, d,X1, X2, census]
varlis = []

# Partial out Variables in varlist from year and county fixed effect
for i in variable
    append!(varlis,i)
end

In [82]:
# Running the following lines takes aprox. 10 minutes (depends on your CPU)

example = DataFrame(CountyCode = data[:,"CountyCode"]);
rdata = DataFrame(CountyCode = data[:,"CountyCode"]);

for i in 1:size(varlis,1)
    rdata[!,varlis[i]]= residuals(lm(term(Symbol(varlis[i])) ~ sum(term.(Symbol.(year))) + sum(term.(Symbol.(fixed))), data))
end

# DML for neural nets

The following algorithm comsumes $Y$,$D$ and $Z$, and learns the residuals $\tilde{Y}$ and $\tilde{D}$ via a neural network, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient β and the clustered standard error from the final OLS regression.

In [67]:
using Flux
using Flux: crossentropy, @epochs
using Flux.Data: DataLoader
using Flux: throttle
using Flux: onehotbatch, onecold, @epochs
using StatsBase

In [50]:
    mean_1 = mean.(eachcol(Z))
    mean_1 = [names(Z) mean_1]

    std_1 = std.(eachcol(Z))
    std_1 = [names(Z) std_1]

    df = DataFrame()
    for i in 1:size(Z)[2]
        p = (Z[!, i] .- mean_1[i,2]) / std_1[i,2]
        colname = names(Z)[i]
        df[!,colname] = p
    end
    Z = df;
    

In [57]:
nfold = 10
    # Num ob observations
nobser = size(z,1)
    
    # Define folds indices 
foldid = collect(Kfold(size(z)[1], nfold));

In [65]:
nfold = 10
    # Num ob observations
nobser = size(z,1)
    
    # Define folds indices 
foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
ytil = ones(nobser)
dtil = ones(nobser)
println("Folds: " )

for i in 1:nfold
    ##############################################
    ################| MODEL D |###################
    model_y= Chain(Dense(size(z,2), 16, relu), 
    Dense(16, 16, relu),
    Dense(16, 1))

    opt = RMSProp()
    loss_y(x, y) = Flux.Losses.mse(model_y(x), y)
    metrics_y(x, y) = Flux.mae(model_y(x), y)
    ps_y = Flux.params(model_y)

    ##############################################
    ################| MODEL Y |###################
    model_d= Chain(Dense(size(z,2), 16, relu), 
    Dense(16, 16, relu),
    Dense(16, 1))

    opt = RMSProp()
    loss_d(x, y) = Flux.Losses.mse(model_d(x), y)
    metrics_d(x, y) = Flux.mae(model_d(x), y)
    ps_d = Flux.params(model_d)

    data_d = DataLoader((z[foldid[i],:]', d[foldid[i]]'))
    data_y = DataLoader((z[foldid[i],:]', y[foldid[i]]'))



    # Lasso regression, excluding folds selected 

    @epochs 100 Flux.train!(loss_y, ps_y, data_y, opt, cb = throttle(() -> println("training"), 10));
    
    @epochs 100 Flux.train!(loss_d, ps_d, data_d, opt, cb = throttle(() -> println("training"), 10));

    # Predict estimates using the 
    yhat = model_y(z[Not(foldid[i]),:]')';
    ###############################################################################
    dhat = model_d(z[Not(foldid[i]),:]')';
    
        
    # Save errors 
    dtil[Not(foldid[i])] = (d[Not(foldid[i])] - dhat)
    ytil[Not(foldid[i])] = (y[Not(foldid[i])] - yhat)

    println(i)

end

    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil, clu=clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil +fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    

Folds: 


┌ Info: Epoch 1
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training
training


┌ Info: Epoch 2
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training


...

training


┌ Info: Epoch 99
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training


┌ Info: Epoch 100
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


10
 coef (se) = 0.06851606922645292([0.05275705396297602])


In [88]:
mean_1 = mean.(eachcol(Z))
    mean_1 = [names(Z) mean_1]

    std_1 = std.(eachcol(Z))
    std_1 = [names(Z) std_1]

    df = DataFrame()
    for i in 1:size(Z)[2]
        p = (Z[!, i] .- mean_1[i,2]) / std_1[i,2]
        colname = names(Z)[i]
        df[!,colname] = p
    end
    Z = df
    z = Matrix(Z);

In [99]:
(z[foldid[1],:]', d[foldid[1]]')

LoadError: BoundsError: attempt to access 1-element Vector{String} at index [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  3891, 3892, 3893, 3894, 3895, 3896, 3897, 3898, 3899, 3900]]

In [107]:
DataLoader((z[foldid[1],:]', d[foldid[1]]'))

DataLoader{Tuple{LinearAlgebra.Adjoint{Float64, Matrix{Float64}}, LinearAlgebra.Adjoint{Float64, Vector{Float64}}}, Random._GLOBAL_RNG}(([0.19297177786972372 0.05136984159535476 … -1.7589560388446375 0.430706036235847; -0.1712110232880095 -0.18550636666572964 … -4.159708239453815 0.23656100088678456; … ; -1.0633826979854069 -1.0633826979853604 … -1.3956164583387598 -1.7806352965715715; 1.692330220407356 1.6923302204073853 … -0.3844641062328095 0.030623506030523804], [0.09612707725792013 0.08080937323687953 … -0.0971269052476953 0.27406158110094614]), 1, 3510, true, 3510, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510], false, Random._GLOBAL_RNG())

In [108]:
function DML2_for_PLM(z , d , y, nfold, clu, num_epochs)
    
    # Num ob observations
    nobser = size(z,1)
    
    # Define folds indices 
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    println("Folds: " )
    


    # loop to save results
    for i in 1:nfold


        ##############################################
        ################| MODEL D |###################
        model_y= Chain(Dense(size(z,2), 16, relu), 
        Dense(16, 16, relu),
        Dense(16, 1))

        opt = RMSProp()
        loss_y(x, y) = Flux.Losses.mse(model_y(x), y)
        metrics_y(x, y) = Flux.mae(model_y(x), y)
        ps_y = Flux.params(model_y)

        ##############################################
        ################| MODEL Y |###################
        model_d= Chain(Dense(size(z,2), 16, relu), 
        Dense(16, 16, relu),
        Dense(16, 1))

        opt = RMSProp()
        loss_d(x, y) = Flux.Losses.mse(model_d(x), y)
        metrics_d(x, y) = Flux.mae(model_d(x), y)
        ps_d = Flux.params(model_d)

        data_d = DataLoader((z[foldid[i],:]', d[foldid[i]]'))
        data_y = DataLoader((z[foldid[i],:]', y[foldid[i]]'))



    # Lasso regression, excluding folds selected 

    @epochs 100 Flux.train!(loss_y, ps_y, data_y, opt, cb = throttle(() -> println("training"), 10));
    
    @epochs 100 Flux.train!(loss_d, ps_d, data_d, opt, cb = throttle(() -> println("training"), 10));

    # Predict estimates using the 
    yhat = model_y(z[Not(foldid[i]),:]')';
    ###############################################################################
    dhat = model_d(z[Not(foldid[i]),:]')';
    
        
    # Save errors 
    dtil[Not(foldid[i])] = (d[Not(foldid[i])] - dhat)
    ytil[Not(foldid[i])] = (y[Not(foldid[i])] - yhat)

    println(i)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil, clu=clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil +fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit, data;
    
end

DML2_for_PLM (generic function with 2 methods)

In [109]:
DML2_for_PLM(z,d,y,10,clu,100)

Folds: 


┌ Info: Epoch 1
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training
training


┌ Info: Epoch 2
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training


...

┌ Info: Epoch 74
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training


┌ Info: Epoch 75
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


training


┌ Info: Epoch 76
└ @ Main C:\Users\PC\.julia\packages\Flux\qp1gc\src\optimise\train.jl:135


# Estimating the effect with DLM for neural nets

In [34]:
# Treatment variable
D = rdata[!,d]

# Outcome variable

Y = rdata[!,y];

# Construct matrix Z
Z = rdata[!, varlis[3:end]];


In [103]:
# Create main variables
z = Matrix(Z);
d = D[!,1];
y = Y[!,1];
clu = rdata[!, :CountyCode];
first(DataFrame(logghomr = y,logfssl = d,CountyCode = clu ),6)

,logghomr,logfssl,CountyCode
,Float64,Float64,Int64
1,-0.134778,0.0961271,1073
2,-0.239622,0.0808094,1073
3,-0.0786772,0.0573399,1073
4,-0.331465,0.0816945,1073
5,-0.31664,0.0253655,1073
6,0.105132,-0.00677726,1073
